In [ ]:
!pip install pdfplumber openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 789.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 51.4 MB/s eta 0:00:00


In [ ]:
import pdfplumber
import re
import pandas as pd
from google.colab import files

#Subida de archivos pdf
uploaded = files.upload()
pdfs = [f for f in uploaded if f.endswith(".pdf")]

Saving NT722_860050906_SECOMN_1507863_PERIODO_202502.pdf to NT722_860050906_SECOMN_1507863_PERIODO_202502.pdf
Saving NT722_860090915_SECOMN_1507992_PERIODO_202502.pdf to NT722_860090915_SECOMN_1507992_PERIODO_202502.pdf
Saving NT722_900276962_SECOMN_1509789_PERIODO_202502.pdf to NT722_900276962_SECOMN_1509789_PERIODO_202502.pdf


In [ ]:
# OTRO INTENTO :'vvv QUE ES EL FINAL

#AFILIADOS IN PAGO-FUNCIONAL SIN CONDICIONAL 0>

# Extraer encabezado general
def extraer_encabezado(texto):
    empresa = re.search(r'NOMBRE\s+EMPRESA\s*:\s*(.+)', texto)
    nit = re.search(r'NIT\s*(\d+)', texto)
    poliza = re.search(r'NUMERO\s+DE\s+POLIZA\s*(\d+)', texto)
    periodo = re.search(r'PERIODO\s+DE\s+COTIZACION\s*(\d+)', texto)
    return {
        'empresa': empresa.group(1).strip() if empresa else "",
        'nit': nit.group(1) if nit else "",
        'poliza': poliza.group(1) if poliza else "",
        'periodo': periodo.group(1) if periodo else ""
    }

# Verifica si una página tiene sección 1
def pagina_con_seccion_1(pagina):
    texto = pagina.extract_text()
    return bool(re.search(r'1\.\s*TRABAJADORES\s+AFILIADOS.*NO\s+PRESENTAN\s+PAGO', texto or "", re.IGNORECASE))

# Regex que separa los campos correctamente
patron_trabajador1 = re.compile(
    r'^(?P<tipo_id>CC|TI|CE|NIT|PT|PE|C)\s+'
    r'(?P<numero_id>\d+)\s+'
    r'(?P<tipo_cotizante>\d+\s-\s\w+)\s+'
    r'(?P<nombre>.*?)\s+(?P<estado>ING|RET|NO\s*SE\s*ENC(?:O|ON|ONT|ONTR|ONTRA|ONTRÓ)?|\bNO\b|No\s*se\s*enco)\s+' # Añadido "No\s*se\s*enco"
    r'(?P<fecha_inicio>\d{2}/\d{2}/\d{4})'
    r'(?:\s+(?P<fecha_fin>\d{2}/\d{2}/\d{4}))?'
    r'\s+(?P<centro_trabajo>\d+\s-\s.+?)'
    r'\s+(?P<tasa>\d+,\d+%|\d+\.\d+%)'
    r'\s+(?P<periodo>\d{6})'
    r'\s+(?P<diferencias>[\d.,]+)$',
    re.IGNORECASE
)

# Lista donde se guarda la info
datos_trabajadores = []

for pdf_file in pdfs:
    with pdfplumber.open(pdf_file) as pdf:
        texto_completo = "\n".join([p.extract_text() or "" for p in pdf.pages])
        encabezado = extraer_encabezado(texto_completo)

        for pagina in pdf.pages:
            if pagina_con_seccion_1(pagina):
                texto = pagina.extract_text()
                for linea in texto.split("\n"):
                    linea_original = linea.strip()
                    linea = linea_original

                    # Normalización del estado "NO SE ENC"
                    linea = re.sub(r'\bNo\s*se\s*enco\b', 'NO SE ENC', linea, re.IGNORECASE) #Más específico

                    # Manejo de nombres faltantes
                    if re.search(r'\b(CC|TI|CE|NIT|PT|PE|C)\s+\d+\s+\d+\s-\s\w+\s+(ING|RET|NO\s*SE\s*ENC|\bNO\b)', linea, re.IGNORECASE):
                        linea = re.sub(
                            r'(\b\d+\s+-\s+\w+)\s+(ING|RET|NO\s*SE\s*ENC|\bNO\b)',
                            r'\1 __SIN_NOMBRE__ \2',
                            linea,
                            re.IGNORECASE
                        )

                    match = patron_trabajador1.search(linea)
                    if match:
                        grupo = match.groupdict()
                        datos_trabajadores.append({
                            'Archivo': pdf_file,
                            'Empresa': encabezado['empresa'],
                            'NIT': encabezado['nit'],
                            'Póliza': encabezado['poliza'],
                            'Período': encabezado['periodo'],
                            'Tipo ID': grupo.get("tipo_id", ""),
                            'Número ID': grupo.get("numero_id", ""),
                            'Tipo de Cotizante': grupo.get("tipo_cotizante", ""),
                            'Nombres y Apellidos': grupo.get("nombre", "").replace("__SIN_NOMBRE__", "").strip(),
                            'Estado de tr': grupo.get("estado", ""),
                            'Fecha Inicio Cobertura': grupo.get("fecha_inicio", ""),
                            'Fecha Fin Cobertura': grupo.get("fecha_fin", ""),  # opcional
                            'Centro Trabajo': grupo.get("centro_trabajo", ""),
                            'Tasa de Cotizacion': grupo.get("tasa", ""),
                            'Período de Mora': grupo.get("periodo", ""),
                            'Diferencias Pendientes': grupo.get("diferencias", "")
                        })

In [ ]:
#SECCION 2

# ===== FUNCIONES COMPARTIDAS =====
def extraer_encabezado(texto):
    empresa = re.search(r'NOMBRE\s+EMPRESA\s*:\s*(.+)', texto)
    nit = re.search(r'NIT\s*(\d+)', texto)
    poliza = re.search(r'NUMERO\s+DE\s+POLIZA\s*(\d+)', texto)
    periodo = re.search(r'PERIODO\s+DE\s+COTIZACION\s*(\d+)', texto)
    return {
        'empresa': empresa.group(1).strip() if empresa else "",
        'nit': nit.group(1) if nit else "",
        'poliza': poliza.group(1) if poliza else "",
        'periodo': periodo.group(1) if periodo else ""
    }

def pagina_con_seccion_2(pagina):
    texto = pagina.extract_text()
    return bool(re.search(r'2\.\s*TRABAJADORES.*NO\s+SE\s+ENCUENTRAN.*AFILIADOS.*ARL.*PRESENTAN\s+PAGO', texto or "", re.IGNORECASE))

# ===== PATRÓN SECCIÓN 2 (Mismo estilo que Sección 1) =====
patron_trabajador2 = re.compile(
    r'^(?P<tipo_id>CC|TI|CE|NIT|PT|PE|C)\s+'
    r'(?P<numero_id>\d+)\s+'
    r'(?P<tipo_cotizante>\d+\s-\s\w+)\s+'
    r'(?P<nombre>.+?)\s+'  # Nombre hasta el siguiente patrón
    r'(?P<tasa>\d+,\d+%|\d+\.\d+%)\s+'
    r'(?P<aporte>[\d\.,]+)$',
    re.IGNORECASE
)

# ===== PROCESAMIENTO (Igual que Sección 1) =====
datos_trabajadores2 = []

for pdf_file in pdfs:
    with pdfplumber.open(pdf_file) as pdf:
        texto_completo = "\n".join([p.extract_text() or "" for p in pdf.pages])
        encabezado = extraer_encabezado(texto_completo)

        for pagina in pdf.pages:
            if pagina_con_seccion_2(pagina):
                texto = pagina.extract_text()
                for linea in texto.split("\n"):
                    linea = linea.strip()

                    # Saltar líneas que no contienen datos (encabezados o vacías)
                    if not linea or re.search(r'TIPO\s*DOCUMENTO|NUMERO\s*IDENTIFICACION', linea, re.IGNORECASE):
                        continue

                    # Normalización básica (sin pipes, espacios múltiples)
                    linea = re.sub(r'[\|]', ' ', linea)  # Eliminar pipes si existen
                    linea = re.sub(r'\s{2,}', ' ', linea)  # Unificar espacios

                    match = patron_trabajador2.search(linea)
                    if match:
                        grupo = match.groupdict()
                        datos_trabajadores2.append({
                            'Archivo': pdf_file,
                            'Empresa': encabezado['empresa'],
                            'NIT': encabezado['nit'],
                            'Póliza': encabezado['poliza'],
                            'Período': encabezado['periodo'],
                            'Tipo Documento': grupo["tipo_id"],
                            'Número Identificación': grupo["numero_id"],
                            'Tipo de Cotizante': grupo["tipo_cotizante"],
                            'Nombres y Apellidos': grupo["nombre"].strip(),
                            'Tasa de Cotización': grupo["tasa"],
                            'Valor Aporte': grupo["aporte"].replace('.', '').replace(',', '.')  # Formato numérico
                        })

In [ ]:
# ====== SECCION 3 =======

def extraer_encabezado(texto):
    empresa = re.search(r'NOMBRE\s+EMPRESA\s*:\s*(.+)', texto)
    nit = re.search(r'NIT\s*(\d+)', texto)
    poliza = re.search(r'NUMERO\s+DE\s+POLIZA\s*(\d+)', texto)
    periodo = re.search(r'PERIODO\s+DE\s+COTIZACION\s*(\d+)', texto)
    return {
        'empresa': empresa.group(1).strip() if empresa else "",
        'nit': nit.group(1) if nit else "",
        'poliza': poliza.group(1) if poliza else "",
        'periodo': periodo.group(1) if periodo else ""
    }

def es_inicio_seccion_3(linea):
    return bool(re.search(r'3\.\s*TRABAJADORES\s+QUE\s+PRESENTAN\s+DIFERENCIAS\s+EN\s+EL\s+PAGO', linea, re.IGNORECASE))

def es_fin_seccion_3(linea):
    return bool(re.search(r'PARA\s+TENER\s+EN\s+CUENTA:\s*Se refiere a los trabajadores que en su pago presentan una tasa', linea, re.IGNORECASE)) or \
           bool(re.search(r'4\.\s*OTRAS\s+DIFERENCIAS', linea, re.IGNORECASE))



# Regex adaptado para capturar los datos de Sección 3
patron_trabajador3 = re.compile(
    r'^(?P<tipo_id>CC|TI|CE|NIT|PT|PE|C)\s+'
    r'(?P<numero_id>\d+)\s+'
    r'(?P<tipo_cotizante>\d+\s-\s\w+)\s+'
    r'(?P<nombre>.*?)\s+'
    r'(?P<estado>ING|RET|NO\s*SE\s*ENC(?:O|ON|ONT|ONTR|ONTRA|ONTRÓ)?|NO)\s*'
    r'(?P<fecha_inicio>\d{2}/\d{2}/\d{4})'
    r'(?:\s+(?P<fecha_fin>\d{2}-[A-Z]{3}-\d{2,4}))?'  # opcional
    r'\s+(?P<centro_trabajo>\d+\s-\s.+?)\s+'
    r'(?P<tasa_bd>\d+[.,]\d+%)\s+'
    r'(?P<tasa_pagada>\d+[.,]\d+%)\s+'
    r'(?P<periodo>\d{6})\s+'
    r'(?P<valor_pagado>[\d.,]+)\s+'
    r'(?P<diferencias>[\d.,]+)$',
    re.IGNORECASE
)


# Lista para guardar datos
datos_trabajadores3 = []

# Procesar archivos
for pdf_file in pdfs:
    with pdfplumber.open(pdf_file) as pdf:
        texto_completo = "\n".join([p.extract_text() or "" for p in pdf.pages])
        encabezado = extraer_encabezado(texto_completo)

        dentro_seccion_3 = False

        for pagina in pdf.pages:
            texto = pagina.extract_text() or ""
            for linea in texto.split("\n"):
                linea = linea.strip()

                if not linea:
                    continue

                # Detectar inicio de sección 3
                if not dentro_seccion_3 and es_inicio_seccion_3(linea):
                    dentro_seccion_3 = True
                    continue

                # Detectar fin de sección 3
                if dentro_seccion_3 and es_fin_seccion_3(linea):
                    dentro_seccion_3 = False
                    break  # Termina de leer esta página

                # Procesar líneas solo si estamos dentro de sección 3
                if dentro_seccion_3:
                    # Saltar encabezados de tablas
                    if re.search(r'TIPO\s*DOCUMENTO|NUMERO\s*IDENTIFICACION|TPO\s*DE\s*COTIZANTE', linea, re.IGNORECASE):
                        continue

                    # Limpiar línea
                    linea = re.sub(r'\bNo\s*se\s*enco\b', 'NO SE ENC', linea, re.IGNORECASE)
                    linea = re.sub(r'\s{2,}', ' ', linea)

                    match = patron_trabajador3.search(linea)
                    if match:
                        grupo = match.groupdict()
                        datos_trabajadores3.append({
                            'Archivo': pdf_file,
                            'Empresa': encabezado['empresa'],
                            'NIT': encabezado['nit'],
                            'Póliza': encabezado['poliza'],
                            'Período': encabezado['periodo'],
                            'Tipo Documento': grupo["tipo_id"],
                            'Número Identificación': grupo["numero_id"],
                            'Tipo de Cotizante': grupo["tipo_cotizante"],
                            'Nombres y Apellidos': grupo["nombre"].strip(),
                            'Estado de tr': grupo["estado"],
                            'Fecha Inicio Cobertura': grupo["fecha_inicio"],
                            'Fecha Fin Cobertura': grupo.get("fecha_fin", ""),
                            'Centro Trabajo': grupo["centro_trabajo"],
                            'Tasa BD': grupo["tasa_bd"],
                            'Tasa Pagada': grupo["tasa_pagada"],
                            'Período Mora': grupo["periodo"],
                            'Valor Pagado': grupo["valor_pagado"].replace('.', '').replace(',', '.'),
                            'Diferencias Pendientes': grupo["diferencias"].replace('.', '').replace(',', '.')
                        })

NO MATCH: 480 $0
NO MATCH: POR TASA DE COTIZACIÓN
NO MATCH: DOCUMENTO IDENTIFICACIÓN Y DESCRIPCIÓN APELLIDOS TRABAJADOR INICIO COBERTURA DESCRIPCIÓN DEL COTIZACIÓN COTIZACIÓN DE PAGO COTIZACIÓN PENDIENTES
NO MATCH: COBERTURA CENTRO BASE DE PAGADO PAGADO
NO MATCH: DATOS
NO MATCH: Page 15 of 364
NO MATCH: CONCILIACIÓN PERIÓDICA DE TRABAJADORES AFILIADOS A LA ARL BOLÍVAR
NO MATCH: DOCUMENTO IDENTIFICACIÓN Y DESCRIPCIÓN APELLIDOS TRABAJADOR INICIO COBERTURA DESCRIPCIÓN DEL COTIZACIÓN COTIZACIÓN DE PAGO COTIZACIÓN PENDIENTES
NO MATCH: COBERTURA CENTRO BASE DE PAGADO PAGADO
NO MATCH: DATOS
NO MATCH: Page 16 of 364
NO MATCH: CONCILIACIÓN PERIÓDICA DE TRABAJADORES AFILIADOS A LA ARL BOLÍVAR
NO MATCH: DOCUMENTO IDENTIFICACIÓN Y DESCRIPCIÓN APELLIDOS TRABAJADOR INICIO COBERTURA DESCRIPCIÓN DEL COTIZACIÓN COTIZACIÓN DE PAGO COTIZACIÓN PENDIENTES
NO MATCH: COBERTURA CENTRO BASE DE PAGADO PAGADO
NO MATCH: DATOS
NO MATCH: Page 17 of 364
NO MATCH: CONCILIACIÓN PERIÓDICA DE TRABAJADORES AFILIADOS A LA

In [ ]:
# ====== SECCION 4 =======

def pagina_con_seccion_4_1(pagina):
    texto = pagina.extract_text()
    return bool(re.search(r'4\.1\s*NOVEDAD\s*DE\s*INGRESO\s*Y\s*RETIRO', texto or "", re.IGNORECASE))

# ===== Verifica si una página tiene sección 4.2 =====
def pagina_con_seccion_4_2(pagina):
    texto = pagina.extract_text()
    return bool(re.search(r'4\.2\s*DIAS\s*COTIZADOS\s*REPORTADOS', texto or "", re.IGNORECASE))


patron_trabajador4_1 = re.compile(
    r'^(?P<tipo_id>CC|TI|CE|NIT|PT|PE|C)\s+'               # Tipo documento
    r'(?P<numero_id>\d+)\s+'                             # Número identificación
    r'(?P<tipo_cotizante>\d+\s-\s[^\d]+?)\s{2,}'          # Tipo cotizante (mínimo 2 espacios)
    r'(?P<nombre>.*?)\s{2,}'                             # Nombre hasta mínimo 2 espacios
    r'(?P<fecha_inicio>\d{2}/\d{2}/\d{4})\s+'             # Fecha inicio cobertura
    r'(?P<centro_trabajo>\d+\s-\s.+?)\s+'                 # Centro de trabajo
    r'(?P<tasa>\d+[,.]\d+%)$',                           # Tasa de cotización
    re.IGNORECASE
)

patron_trabajador4_2 = re.compile(
    r'^(?P<tipo_id>CC|TI|CE|NIT|PT|PE|C)\s+'               # Tipo documento
    r'(?P<numero_id>\d+)\s+'                             # Número identificación
    r'(?P<tipo_cotizante>\d+\s-\s[^\d]+?)\s+'             # Tipo cotizante (puede ser 1 espacio)
    r'(?P<nombre>.*?)\s+'                                # Nombre hasta 1 espacio
    r'(?P<fecha_inicio>\d{2}/\d{2}/\d{4})\s+'
    r'(?P<centro_trabajo>\d+\s-\s.+?)\s+'
    r'(?P<tasa>\d+[,.]\d+%)$',
    re.IGNORECASE
)

# Inicializar listas
datos_trabajadores4_1 = []
datos_trabajadores4_2 = []

for pdf_file in pdfs:
    with pdfplumber.open(pdf_file) as pdf:
        texto_completo = "\n".join([p.extract_text() or "" for p in pdf.pages])
        encabezado = extraer_encabezado(texto_completo)

        for pagina in pdf.pages:
            texto = pagina.extract_text()
            if not texto:
                continue

            if pagina_con_seccion_4_1(pagina):
                for linea in texto.split("\n"):
                    linea = linea.strip()
                    match = patron_trabajador4_1.search(linea)
                    if match:
                        grupo = match.groupdict()
                        datos_trabajadores4_1.append({
                            'Archivo': pdf_file,
                            'Empresa': encabezado['empresa'],
                            'NIT': encabezado['nit'],
                            'Póliza': encabezado['poliza'],
                            'Período': encabezado['periodo'],
                            'Tipo ID': grupo.get("tipo_id", ""),
                            'Número ID': grupo.get("numero_id", ""),
                            'Tipo de Cotizante': grupo.get("tipo_cotizante", ""),
                            'Nombres y Apellidos': grupo.get("nombre", ""),
                            'Fecha Inicio Cobertura': grupo.get("fecha_inicio", ""),
                            'Centro Trabajo': grupo.get("centro_trabajo", ""),
                            'Tasa de Cotizacion': grupo.get("tasa", "")
                        })

            if pagina_con_seccion_4_2(pagina):
                for linea in texto.split("\n"):
                    linea = linea.strip()
                    match = patron_trabajador4_2.search(linea)
                    if match:
                        grupo = match.groupdict()
                        datos_trabajadores4_2.append({
                            'Archivo': pdf_file,
                            'Empresa': encabezado['empresa'],
                            'NIT': encabezado['nit'],
                            'Póliza': encabezado['poliza'],
                            'Período': encabezado['periodo'],
                            'Tipo ID': grupo.get("tipo_id", ""),
                            'Número ID': grupo.get("numero_id", ""),
                            'Tipo de Cotizante': grupo.get("tipo_cotizante", ""),
                            'Nombres y Apellidos': grupo.get("nombre", ""),
                            'Fecha Inicio Cobertura': grupo.get("fecha_inicio", ""),
                            'Centro Trabajo': grupo.get("centro_trabajo", ""),
                            'Tasa de Cotizacion': grupo.get("tasa", "")
                        })

In [ ]:
# ====== SECCION 5 =======


# Función para extraer encabezado
def extraer_encabezado(texto):
    empresa = re.search(r'NOMBRE\s+EMPRESA\s*:\s*(.+)', texto)
    nit = re.search(r'NIT\s*(\d+)', texto)
    poliza = re.search(r'NUMERO\s+DE\s+POLIZA\s*(\d+)', texto)
    periodo = re.search(r'PERIODO\s+DE\s+COTIZACION\s*(\d+)', texto)
    return {
        'empresa': empresa.group(1).strip() if empresa else "",
        'nit': nit.group(1) if nit else "",
        'poliza': poliza.group(1) if poliza else "",
        'periodo': periodo.group(1) if periodo else ""
    }

# Función para detectar si una página contiene la sección 5
def pagina_con_seccion_5(pagina):
    texto = pagina.extract_text().replace("\n", " ")
    return bool(re.search(r'5\.\s*ESTADO\s+ACTUAL\s+BASE\s+DE\s+DATOS\s+DE\s+TRABAJADORES.*AFILIADOS.*ARL', texto or "", re.IGNORECASE))

# ===== REGEX =====
# Patrón ajustado para capturar los datos de trabajadores en la sección 5
patron_trabajador5 = re.compile(
    r'^(?P<tpo_id>\S+)\s+'  # Tipo de documento (CC, CE, etc.)
    r'(?P<nro_id>\d+)\s+'     # Número de identificación
    r'(?P<tipo_cotizante>\d+\s[^\d]+)\s+'  # Tipo de cotizante y descripción
    r'(?P<nombre>[A-ZÑÁÉÍÓÚÜ]+(?:\s[A-ZÑÁÉÍÓÚÜ]+)+)\s+'  # Nombres y apellidos
    r'(?P<fecha_inicio>\d{2}/\d{2}/\d{4})\s+'  # Fecha de inicio cobertura
    r'(?P<centro_trabajo>.+)$',  # Centro de trabajo y descripción
    re.IGNORECASE
)

# ===== LISTA FINAL =====
datos_trabajadores5 = []

# ===== PROCESAR PDFs =====
for pdf_file in pdfs:
    with pdfplumber.open(pdf_file) as pdf:
        # Extraer texto completo para encabezados
        texto_completo = "\n".join([p.extract_text() or "" for p in pdf.pages])
        encabezado = extraer_encabezado(texto_completo)

        # Procesar cada página
        for pagina in pdf.pages:
            if pagina_con_seccion_5(pagina):
                texto = pagina.extract_text()
                seccion_encontrada = False
                for linea in texto.split("\n"):
                    linea = linea.strip()

                    # Detectar el inicio de la lista de trabajadores en la sección 5
                    if re.match(r'^TPO\s+NRO\s+DE\s+TIPO\s+DE\s+.*DESCRIPCIÓN.*', linea):
                        seccion_encontrada = True
                        continue  # Saltar el encabezado de la tabla

                    # Ignorar líneas si no estamos en la sección 5
                    if not seccion_encontrada:
                        continue

                    # Depuración: imprimir líneas que no hacen match
                    match = patron_trabajador5.search(linea)
                    # Si la línea hace match, agregar los datos
                    if match:
                        grupo = match.groupdict()
                        datos_trabajadores5.append({
                            'Archivo': pdf_file,
                            'Empresa': encabezado['empresa'],
                            'NIT': encabezado['nit'],
                            'Póliza': encabezado['poliza'],
                            'Período': encabezado['periodo'],
                            'Tipo Documento': grupo.get("tpo_id", ""),
                            'Número Identificación': grupo.get("nro_id", ""),
                            'Tipo de Cotización y Descripción': grupo.get("tipo_cotizante", ""),
                            'Nombres y Apellidos': grupo.get("nombre", ""),
                            'Fecha Inicio Cobertura': grupo.get("fecha_inicio", ""),
                            'Centro de Trabajo y Descripción del Centro': grupo.get("centro_trabajo", "")
                        })

NO MATCH: DOCUMENTO IDENTIFICACIÓN COBERTURA CENTRO
NO MATCH: Page 22 of 364
NO MATCH: DOCUMENTO IDENTIFICACIÓN COBERTURA CENTRO
NO MATCH: Page 23 of 364
NO MATCH: DOCUMENTO IDENTIFICACIÓN COBERTURA CENTRO
NO MATCH: Page 24 of 364
NO MATCH: DOCUMENTO IDENTIFICACIÓN COBERTURA CENTRO
NO MATCH: Page 25 of 364
NO MATCH: DOCUMENTO IDENTIFICACIÓN COBERTURA CENTRO
NO MATCH: Page 26 of 364
NO MATCH: DOCUMENTO IDENTIFICACIÓN COBERTURA CENTRO
NO MATCH: Page 27 of 364
NO MATCH: DOCUMENTO IDENTIFICACIÓN COBERTURA CENTRO
NO MATCH: Page 28 of 364
NO MATCH: DOCUMENTO IDENTIFICACIÓN COBERTURA CENTRO
NO MATCH: Page 29 of 364
NO MATCH: DOCUMENTO IDENTIFICACIÓN COBERTURA CENTRO
NO MATCH: Page 30 of 364
NO MATCH: DOCUMENTO IDENTIFICACIÓN COBERTURA CENTRO
NO MATCH: Page 31 of 364
NO MATCH: DOCUMENTO IDENTIFICACIÓN COBERTURA CENTRO
NO MATCH: Page 32 of 364
NO MATCH: DOCUMENTO IDENTIFICACIÓN COBERTURA CENTRO
NO MATCH: Page 33 of 364
NO MATCH: DOCUMENTO IDENTIFICACIÓN COBERTURA CENTRO
NO MATCH: Page 34 of 364

In [ ]:
# Exportar a Excel1
with pd.ExcelWriter("conciliaciones_pdfs.xlsx") as writer:
    if datos_trabajadores:
        pd.DataFrame(datos_trabajadores).to_excel(writer, sheet_name="Sección 1", index=False)
    if datos_trabajadores2:
        pd.DataFrame(datos_trabajadores2).to_excel(writer, sheet_name="Sección 2", index=False)
    if datos_trabajadores3:
        pd.DataFrame(datos_trabajadores3).to_excel(writer, sheet_name="Sección 3", index=False)
    if datos_trabajadores4_1:
        pd.DataFrame(datos_trabajadores4_1).to_excel(writer, sheet_name="Sección 4.1", index=False)
    if datos_trabajadores4_2:
        pd.DataFrame(datos_trabajadores4_2).to_excel(writer, sheet_name="Sección 4.2", index=False)
    if datos_trabajadores5:
        pd.DataFrame(datos_trabajadores5).to_excel(writer, sheet_name="Sección 5", index=False)

files.download("conciliaciones_pdfs.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>